In [2]:
!pip install cython

     |████████████████████████████████| 1.9 MB 1.4 MB/s eta 0:00:01


In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython -a
cimport numpy as np

In [6]:
%%cython -a
cdef int measurements_number = 2 ** 10  # number of temperature points
cdef int lattice_size = 10  # size of the lattice, N x N
cdef int equilibration_steps = 2 ** 10  # number of MC sweeps for equilibration
cdef int calculation_steps = 2 ** 10  # number of MC sweeps for calculation

cdef float denominator_1 = 1.0 / (calculation_steps * lattice_size * lattice_size)
cdef float denominator_2 = 1.0 / (
        calculation_steps * calculation_steps * lattice_size * lattice_size)
cdef float critical_temperature = 2.269
cdef int interaction_energy = 1  # J
cdef int wolffs_epochs = 500
cdef int sw_iterations = 35
cdef int bc_simulation = 300
cdef (int, int) lattice_dim = (lattice_size, lattice_size)

In [62]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np
import random

# cnp.import_array()  # needed to initialize numpy-API

cdef int lattice_size = 10
cdef (int, int) lattice_dim = (lattice_size, lattice_size)

state = np.random.choice([1,-1], lattice_size*lattice_size)
    
    
neighbours = tabulate_neighbors(lattice_dim)

@cython.wraparound(False)
@cython.boundscheck(False)
cdef int get_site((int, int) coord, (int, int) L):
    return coord[1] * L[0] + coord[0]

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.cdivision(True)
cdef (int, int) get_coord(int site, (int, int) L):
    cdef int y
    cdef int x
    """Get the 2-vector of coordinates from the site index."""
    if site // L[0] == 0:
        y = site // L[0] - 1
        x = site - (L[0] * y)
        return x, y
    else:
        y = site // (L[0])
        x = site - (L[0] * y)
    return x, y

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.cdivision(True)
cdef long[:] get_neighbors(int site, (int,int) L):
    cdef int y, y1
    cdef int x, x1
    cdef np.ndarray[np.int_t, ndim=1] neighb
    neighb = np.arange(4, dtype=np.int)
    x, y = get_coord(site, L)
    cdef list neighb_coords = [(-1, 0), (1, 0), (0, 1), (0, -1)]
    for each in range(4):
        x1 = (x + neighb_coords[each][0]) % L[0]
        y1 = (y + neighb_coords[each][1]) % L[1]
        neighb[each] = (get_site([x1, y1], L))

    return neighb

@cython.wraparound(False)
@cython.boundscheck(False)
cdef long[:,::1] tabulate_neighbors((int, int) L):
    cdef long[:,::1] neighb = np.empty((lattice_size * lattice_size, 4), dtype=int)
    for site in range(lattice_size * lattice_size):
        neighb[site, :] = get_neighbors(site, L)
    return neighb


cdef int interaction_energy = 1  # J


@cython.wraparound(False)
@cython.boundscheck(False)
cdef tuple SW_BFS(bonded, dict clusters, tuple start, double beta, int nearest_neighbors=1):
    """
    :param bonded: 1 or 0, indicates whether a site has been assigned to a cluster
           or not
    :param clusters: dictionary containing all existing clusters, keys are an integer
            denoting natural index of root of cluster
    :param start: root node of graph (x,y)
    :param beta: temperature
    :param nearest_neighbors: number or NN to probe
    :return:
    """
    cdef long[:] visited = np.empty(lattice_size ** 2)  # indexes whether we have visited nodes during
    # this particular BFS search
    if bonded[start] != 0:  # cannot construct a cluster from this site
        return bonded, clusters, visited

    p = 1 - np.exp(-2 * beta * interaction_energy)  # bond forming probability

    queue = [start]
    index = start
    clusters[index] = [index]
    cluster_spin = state[index]
    color = np.max(bonded) + 1

    # whatever the input coordinates are
    while len(queue) > 0:
        r = queue.pop(0)
        if visited[r] == 0:  # if not visited
            visited[r] = 1
            # to see clusters, always use different numbers
            bonded[r] = color
            NN = neighbours[r]
            for nn_coords in NN:
                if state[nn_coords] == cluster_spin and bonded[nn_coords] == 0 and visited[nn_coords] == 0:
                    random_val = np.random.rand()
                    if random_val < p:  # accept bond proposal
                        queue.append(nn_coords)  # add coordinate to search
                        clusters[index].append(nn_coords)  # add point to the cluster
                        bonded[nn_coords] = color  # indicate site is no longer available

    return bonded, clusters, visited

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.cdivision(True)
cdef void sw_move(double temp):
    cdef np.ndarray[np.int_t, ndim=1] bonded = np.arrange(lattice_size ** 2)
    beta = 1.0 / temp
    clusters = dict()  # keep track of bonds

    for i in range(lattice_size):
        for j in range(lattice_size):
            coord = get_site([i, j], lattice_dim)
            bonded, clusters, visited = SW_BFS(bonded, clusters, coord, beta,
                                                    nearest_neighbors=1)

    for cluster_index in clusters.keys():
        r = np.random.rand()
        if r < 0.5:
            for coords in clusters[cluster_index]:
                state[coords] = -1 * state[coords]